In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')

import PETsARD

In [2]:
loader = PETsARD.Loader(
    filepath='benchmark://adult',
    na_values={k: '?' for k in [
        'workclass',
        'occupation',
        'native-country'
    ]}
)

splitter = PETsARD.Splitter(
    data=loader.data,
    num_samples=30,
    train_split_ratio=0.8
)

preproc = PETsARD.Preprocessor(
    data=splitter.data[1]['train'],
    missing=True,
    missing_method='drop',
    missing_columns=None,
    outlier=True,
    outlier_method='IQR',
    outlier_columns={'ignore': ['hours-per-week']},
    encoding=True,
    encoding_method='Label',
    encoding_columns=None,
    scaling=True,
    scaling_method='Standard',
    scaling_columns={'focus': 'fnlwgt'}
)

synthesizer = PETsARD.Synthesizer(
    data=preproc.data,
    synthesizing_method='sdv-singletable-gaussiancopula'
)
synthesizer.fit_sample()

postproc = PETsARD.Postprocessor(
    data=synthesizer.data_syn,
    missingist = getattr(preproc ,'missingist' ,None),
    encoder=getattr(preproc, 'encoder', None),
    scaler=getattr(preproc, 'scaler', None)
)
postproc.data.head(1)

Loader - Benchmarker: file benchmark\adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
Preprocessor - Outlierist (IQR): Dropped  1066 rows on fnlwgt         . Kept [-62937.0, 417893.0] only.
Preprocessor - Outlierist (IQR): Dropped   213 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  3025 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1703 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   229 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6039 in 36150 rows.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column native-country  been labelized from 0 to 39.
Preprocessor - Encoder (Label): Column occupation      been labelized from 0 to 13.
Preprocessor - Encoder (Label): Col

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,50.0,Private,179966.726791,Masters,10.0,Divorced,Sales,Unmarried,White,Male,0.0,0.0,35.0,Yugoslavia,<=50K


In [6]:
eval = PETsARD.Evaluator(
    evaluating_method='sdmetrics-single_table-diagnosticreport',
    data={
        'ori': splitter.data[1]['train'],
        'syn': postproc.data,
        'control': splitter.data[1]['validation']
    }
)
eval.eval()
eval.Evaluator.evaluation

Evaluator (SDMetrics): Evaluating DiagnosticReport.
Generating report ...
(2/2) Evaluating Data Structure: : 100%|██████████| 1/1 [00:00<00:00, 500.04it/s]

Overall Score: 100.0%

Properties:
- Data Validity: 100.0%
- Data Structure: 100.0%
Evaluator (SDMetrics): Evaluating DiagnosticReport spent 0.1461 sec.


{'score': 1.0,
 'properties': {'Data Validity': {'Score': 1.0},
  'Data Structure': {'Score': 1.0}},
 'details': {'Data Validity':              Column             Metric  Score
  0               age  BoundaryAdherence    1.0
  1         workclass  CategoryAdherence    1.0
  2            fnlwgt  BoundaryAdherence    1.0
  3         education  CategoryAdherence    1.0
  4   educational-num  BoundaryAdherence    1.0
  5    marital-status  CategoryAdherence    1.0
  6        occupation  CategoryAdherence    1.0
  7      relationship  CategoryAdherence    1.0
  8              race  CategoryAdherence    1.0
  9            gender  CategoryAdherence    1.0
  10     capital-gain  BoundaryAdherence    1.0
  11     capital-loss  BoundaryAdherence    1.0
  12   hours-per-week  BoundaryAdherence    1.0
  13   native-country  CategoryAdherence    1.0
  14           income  CategoryAdherence    1.0,
  'Data Structure':            Metric  Score
  0  TableStructure    1.0}}

In [5]:
eval = PETsARD.Evaluator(
    evaluating_method='sdmetrics-single_table-qualityreport',
    data={
        'ori': splitter.data[1]['train'],
        'syn': postproc.data,
        'control': splitter.data[1]['validation']
    }
)
eval.eval()
eval.Evaluator.evaluation

Evaluator (SDMetrics): Evaluating QualityReport.
Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 105/105 [00:06<00:00, 16.37it/s]

Overall Score: 65.43%

Properties:
- Column Shapes: 83.91%
- Column Pair Trends: 46.94%
Evaluator (SDMetrics): Evaluating QualityReport spent 6.6671 sec.


{'score': 0.6542589328656804,
 'properties': {'Column Shapes': {'Score': 0.8390805399041777},
  'Column Pair Trends': {'Score': 0.46943732582718295}},
 'details': {'Column Shapes':              Column        Metric     Score
  0               age  KSComplement  0.971979
  1         workclass  TVComplement  0.989909
  2            fnlwgt  KSComplement  0.950034
  3         education  TVComplement  0.544262
  4   educational-num  KSComplement  0.869881
  5    marital-status  TVComplement  0.976700
  6        occupation  TVComplement  0.641634
  7      relationship  TVComplement  0.971057
  8              race  TVComplement  0.997578
  9            gender  TVComplement  0.987255
  10     capital-gain  KSComplement  0.917411
  11     capital-loss  KSComplement  0.953523
  12   hours-per-week  KSComplement  0.775636
  13   native-country  TVComplement  0.080731
  14           income  TVComplement  0.958619,
  'Column Pair Trends':            Column 1         Column 2                 Metric 

In [9]:
# https://docs.sdv.dev/sdmetrics/
# https://github.com/sdv-dev/SDMetrics
from sdv.metadata import SingleTableMetadata
import pprint

real_data_metadata = SingleTableMetadata()
real_data_metadata.detect_from_dataframe(splitter.data[1]['train'])
real_data_metadata = real_data_metadata.to_dict()

pprint.pprint(real_data_metadata)


{'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1',
 'columns': {'age': {'sdtype': 'numerical'},
             'capital-gain': {'sdtype': 'numerical'},
             'capital-loss': {'sdtype': 'numerical'},
             'education': {'sdtype': 'categorical'},
             'educational-num': {'sdtype': 'numerical'},
             'fnlwgt': {'sdtype': 'numerical'},
             'gender': {'sdtype': 'categorical'},
             'hours-per-week': {'sdtype': 'numerical'},
             'income': {'sdtype': 'categorical'},
             'marital-status': {'sdtype': 'categorical'},
             'native-country': {'sdtype': 'categorical'},
             'occupation': {'sdtype': 'categorical'},
             'race': {'sdtype': 'categorical'},
             'relationship': {'sdtype': 'categorical'},
             'workclass': {'sdtype': 'categorical'}}}


In [10]:
from sdmetrics.reports.single_table import QualityReport

qlt_report = QualityReport()
qlt_report.generate(
    real_data = splitter.data[1]['train'],
    synthetic_data = postproc.data,
    metadata = real_data_metadata
)
print(f"\n\n")
print(f"get_score: {qlt_report.get_score()}")
print(f"\n\n")
print(qlt_report.get_properties())
print(f"\n\n")
print(qlt_report.get_details(property_name='Column Shapes'))
print(f"\n\n")
print(qlt_report.get_details(property_name='Column Pair Trends'))

Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 105/105 [00:06<00:00, 17.14it/s]

Overall Quality Score: 66.19%

Properties:
- Column Shapes: 84.21%
- Column Pair Trends: 48.17%



get_score: 0.6619173316641179



             Property     Score
0       Column Shapes  0.842111
1  Column Pair Trends  0.481723



             Column        Metric     Score
0               age  KSComplement  0.970691
1         workclass  TVComplement  0.986822
2            fnlwgt  KSComplement  0.954944
3         education  TVComplement  0.546924
4   educational-num  KSComplement  0.871966
5    marital-status  TVComplement  0.977959
6        occupation  TVComplement  0.673373
7      relationship  TVComplement  0.979580
8              race  TVComplement  0.995146
9            gender  TVComplement  0.987174
10     capital-gain  KSComplement  0.916925
11     capital-loss  KSComplement  0.953830
12   hours-per-week  KSComplement  0.771045
13   native-country  TVComplement  0.0833

In [14]:
fig = qlt_report.get_visualization(property_name='Column Shapes')
fig.show()

In [15]:
fig = qlt_report.get_visualization(property_name='Column Pair Trends')
fig.show()

In [6]:
from sdmetrics.reports.single_table import DiagnosticReport

dx_report = DiagnosticReport()
dx_report.generate(
    real_data = splitter.data[1]['train'],
    synthetic_data = postproc.data,
    metadata = real_data_metadata
)
print(f"\n\n")
print(f"get_score: {dx_report.get_score()}")
print(f"\n\n")
print(dx_report.get_properties())
print(f"\n\n")
print(dx_report.get_details(property_name='Data Validity'))
print(f"\n\n")
print(dx_report.get_details(property_name='Data Structure'))


Generating report ...
(2/2) Evaluating Data Structure: : 100%|██████████| 1/1 [00:00<?, ?it/s]

Overall Score: 100.0%

Properties:
- Data Validity: 100.0%
- Data Structure: 100.0%



get_score: 1.0



         Property  Score
0   Data Validity    1.0
1  Data Structure    1.0



             Column             Metric  Score
0               age  BoundaryAdherence    1.0
1         workclass  CategoryAdherence    1.0
2            fnlwgt  BoundaryAdherence    1.0
3         education  CategoryAdherence    1.0
4   educational-num  BoundaryAdherence    1.0
5    marital-status  CategoryAdherence    1.0
6        occupation  CategoryAdherence    1.0
7      relationship  CategoryAdherence    1.0
8              race  CategoryAdherence    1.0
9            gender  CategoryAdherence    1.0
10     capital-gain  BoundaryAdherence    1.0
11     capital-loss  BoundaryAdherence    1.0
12   hours-per-week  BoundaryAdherence    1.0
13   native-country  CategoryAdherence    1.0
14           income  CategoryAd